<h1><center> Lab Session n°2 : RL for stochastic control problems  </center></h1>



<h2> 📌 Objectives: </h2>

In the **first part** of the **lab session**, you will solve one stochastic contol problem in finance : A simple market impact model.  You will explore and implement the algorithms introduced in the lecture using Python and its scientific libraries.

The **second part** of the **lab session** will be devoted to mathematical questions on solving linear quadratic control problems in continuous time.


<h2>📚 Goal of the Lab: </h2>

By the end of this lab, you will be able to:

- Undertand and implement some **reinforcement algorithms** to solve stochastic control problems arising in finance.

- Derive explicitly the **optimal policy** and **value function** arising in the **Linear quadratic** control problems in continuous time.


<h2> 🗂️ Lab Structure and assignments: </h2>

This notebook is organized into the following sections:




**1. [On the Market impact problem](#MarketImpact-Applications)**

&nbsp;&nbsp;&nbsp;&nbsp;1.1 [Problem formulation](#MarketImpact-problemformulation)

&nbsp;&nbsp;&nbsp;&nbsp;1.2 [Q-learning and Sarsa algorithms](#MarketImpact-Q-function)


**2. [Mathematical questions](#Mathematical-Questions)**

&nbsp;&nbsp;&nbsp;&nbsp; [Linear quadratic case in continuous time](#Mathematical-Questions-LQ-case)




**3. [References](#references)**  


This lab will include **mathematics** and/or **coding** questions indicated by ❓. **Your answers** indicated by ✏️ will count for your final grade of the course, with a weight to be determined later with respect to the project. Note that the project will have a significant higher weight in the final grade.


**Mathematics Questions**

- You can answer directly in the **Jupyter notebook** using LaTeX (compatible with Markdown).


**Coding Questions**

-  Complete the corresponding code sections **directly in the notebook**.
-  **Code readability**, **quality**, and **clarity of comments** will be taken into account in the **grading**.


If you choose this lab, you will have to send your work by e-mail at [samy.mekkaoui@polytechnique.edu](mailto:samy.mekkaoui@polytechnique.edu). The submission deadline will be announced later during the course.



<h2>ℹ️ Other informations: </h2>




- **Key References**: If you want to go deeper on the use of RL methods for solving stochastic control problems in finance, you can look at the section [References](#references). <br> <br>



- **Contact**: If you find any mistakes in this notebook, or have any other feedback or questions, please feel free to e-mail me at [samy.mekkaoui@polytechnique.edu](mailto:samy.mekkaoui@polytechnique.edu).


In [8]:
# Import Packages

import numpy as np
import matplotlib
import pandas as pd

from scipy.stats import norm

import matplotlib.pyplot as plt 

from tqdm import tqdm 

import time

<a id=MarketImpact-Applications></a>

<h1> <center> 1: On the Market Impact Problem  </center> </h1>



<a  id=MarketImpact-problemformulation></a>

<h2> 1.1 Problem formulation : </h2>

Assume that the broker has to sell $N$ blocks of shares with $n$ shares in each block.  


In each step, the agent has four possible actions $a_t= a^{(i)}$ where $ i \in \lbrace 0, 1, 2 ,3 \rbrace$ and $a^{(i)}=i$ measure the number of blocks of shares sold at time $t$. The updated equation for the stock number of blocks is then given by

$$
\begin{align}
X_{t+1} = (X_t-a_t)_{+}
\end{align}
$$

The trades influence the stock price dynamics through a linear market impact as

$$
\begin{align}
S_{t+1} = S_t e^{(1- \nu)a_t)} + \sigma S_t \epsilon_t,
\end{align}
$$
where $\nu$ represents a market impact parameter.

The reward function is given by

$r_t= n a_t S_t - \lambda n \text{ Var }[ S_{t+1} X_{t+1} ]$





In [ ]:
BLOCK_SIZE = 1000
NUM_BLOCKS = 10
NUM_S      = 12  #number of discrete values of S
NUM_TIME_STEPS = 10
dt         = 1 # time step
sigma      = 0.1 # volatility of the stock
nu         = 1 # market impact parameter
S0         = 1 # initial stock price
lmbda      = 0.01 # risk aversion parameter

In [9]:
epsilon = 0.1# Probability for exploration
eta = 0.5# Step size

In the list below, we define 

- The potential actions values, i.e., the number of stocks to sell in the current time and state.
- The state vector for the initial state : [number of blocks of shares, stock price, time]
  

In [ ]:
ACTIONS = [0, 1, 2, 3]
START = [NUM_BLOCKS - 1, S0, 0]




 ❓ **Question 1.1**: Fill in the function **transition** below the next state and reward obtained from taking action $a$ in state $s$ at time $t$.

In [ ]:
def transition(
    state,   # list: [inventory, price_state, time]
    action,  # int: number of blocks sold
):
    """
    One-step transition for an optimal execution MDP.

    Parameters
    ----------
    state : list of length 3
        state[0] : inventory (number of blocks)
        state[1] : price state (discretized)
        state[2] : time

    action : int
        Number of blocks sold at the current time step

    Returns
    -------
    (next_state, reward) :  State at the next time step and immediate reward
    """

    X = state[0]   # inventory
    S = state[1]   # discretized price state
    t = state[2]   # time

     # You can't sell more stock than you have
    if action > X: 
        action = X

    X_next = ### 

    t_next = t + dt
 
    S_next = (
        S * np.exp(1 - nu * action)
        + sigma * S * np.sqrt(dt) * np.random.randn()
    )
    S_next = int(np.clip(np.ceil(S_next), 0, NUM_S - 1))

    next_state = [X_next, S_next,t + dt]
   
    reward = ###

    return next_state, reward



 ❓ **Question 1.2**: Implement in the function **epsilon_greedy** below the $\epsilon$-greedy strategy.

In [ ]:
# Epsilon-greedy policy for action selection
def epsilon_greedy(
    state,                      # Current state [inventory, price_state, time]
    q_value,                    # Q-value table Q(s,a)
    eps=epsilon                 # Exploration rate ε
):
    """
    Returns:
        action: integer action selected according to ε-greedy policy
    """

    # --- Step 1: Exploration vs exploitation decision ---
    # Draw a Bernoulli(ε) random variable
    if ______________________________:
        action = ______________________

    # --- Step 2: Greedy action selection ---
    # Extract Q-values at current state and select a maximizer
    else:
        values_ = ______________________
        action = ______________________

    # --- Step 3: Enforce inventory constraint ---
    # Ensure action does not exceed available inventory
    if ______________________________:
        action = ______________________

    return action


<a id=MarketImpact-Q-function></a>

<h2> 1.2 Q-learning and Sarsa algorithms : </h2>

In [ ]:
def compare_sarsa_qlearning(
    num_episodes=1000,
    num_runs=100,
    epoch_size=25
):
    """
    Compare SARSA and Q-learning by averaging episode returns
    over multiple independent runs.
    """

    # --------------------------------------------------
    # Initialize arrays to store average episode rewards
    # --------------------------------------------------
    avg_rewards_sarsa = np.zeros(num_episodes)
    avg_rewards_qlearning = np.zeros(num_episodes)

    # --------------------------------------------------
    # Loop over independent runs
    # --------------------------------------------------
    for run_id in tqdm(range(num_runs)):

        # Initialize Q-tables
        q_table_sarsa = np.zeros(
            (NUM_BLOCKS, NUM_S, NUM_TIME_STEPS, len(ACTIONS))
        )
        q_table_qlearning = np.copy(q_table_sarsa)

        # --------------------------------------------------
        # Loop over episodes
        # --------------------------------------------------
        for episode in range(num_episodes):

            # Epsilon decay schedule (piecewise exponential)
            eps = epsilon * ((1 - epsilon) ** (episode // epoch_size))

            avg_rewards_sarsa[episode] += sarsa(
                q_table_sarsa, eps=eps
            )
            avg_rewards_qlearning[episode] += q_learning(
                q_table_qlearning, eps=eps
            )

    # --------------------------------------------------
    # Average over runs
    # --------------------------------------------------
    avg_rewards_sarsa /= num_runs
    avg_rewards_qlearning /= num_runs

    # --------------------------------------------------
    # Plot learning curves
    # --------------------------------------------------
    plt.plot(avg_rewards_sarsa, label="SARSA")
    plt.plot(avg_rewards_qlearning, label="Q-learning")
    plt.xlabel("Episodes")
    plt.ylabel("Average return per episode")
    plt.legend()

    return q_table_sarsa, q_table_qlearning



 ❓ **Question 1.5**: Comment the plots obtained for the Q-learning and Sarsa algorithms. Compare the two algorithms.

<a id=MarketImpact-Applications></a>





 ❓ **Question 1.6**: Fill the function **optimal_policy** to compute the optimal policy from the learned Q-function.

In [ ]:
def print_optimal_policy(q_value):
    """
    Display the greedy policy associated with a learned Q-table.

    Parameters
    ----------
    q_value : array-like
        Q-value table of shape (NUM_BLOCKS, NUM_S, NUM_TIME_STEPS, |A|)
    """

    # --------------------------------------------------
    # Step 1: Initialize the policy container
    # --------------------------------------------------
    optimal_policy = ______________________________

    # --------------------------------------------------
    # Step 2: Extract greedy action at each state
    # --------------------------------------------------
    for i in range(NUM_BLOCKS):
        for j in range(NUM_S):
            for k in range(NUM_TIME_STEPS):
                ______________________________
                optimal_policy[i, j, k] = ______________________________

    # --------------------------------------------------
    # Step 3: Display policy by time slice
    # --------------------------------------------------
    for k in range(NUM_TIME_STEPS):
        print("========= time step " + str(k) + " ======")
        print(" price: 1,2,3,4,5,6,7,8,9,10,11,12")

        for i in range(NUM_BLOCKS):
            row_str = ______________________________
            for j in range(NUM_S):
                row_str += ______________________________
            print(row_str)


In [ ]:

 ❓ **Question 1.6**: Fill the function **optimal_policy** to compute the optimal policy from the learned Q-function.


 ❓ **Question 1.7**: Print the optimal policy obtained from the Q-learning and Sarsa algorithms.

<a id=Mathematical-Questions></a>
<h1> <center> 3. Mathematical Questions </center> </h1>

We recall that given a policy $\pi = (\pi_s)_{t \leq s \leq T}$, i.e., a $\mathcal{P}(A)$-valued map,  the value function $V^{\pi}$ of the stochastic control problem is defined as

$$
\begin{align}
V^{\pi}(t,x) &= \mathbb{E}_{\pi} \Big[ \int_t^T f(s,X_s,\pi_s) ds + \lambda \mathcal{E}(\pi(s,X_s))  g(X_T) \big | X_t = x \Big], \\
&= \mathbb{E}_{\pi} \Big[ \int_t^T f(s,X_s,\pi_s) ds - \lambda \text{log} (\pi(s,X_s,\alpha_s))  g(X_T) \big | X_t = x \Big]
\end{align}
$$
where the controlled state process is given for $\alpha=(\alpha_s)_{t \leq s \leq T} \sim \pi$ by 
$$
\begin{align}
\begin{cases}
dX_s &= b(X_s, \alpha_s) ds + \sigma(X_s, \alpha_s) dW_s, \quad s \in [t,T], \notag \\
X_t &= x, \notag
\end{cases}
\end{align}
$$
and the optimal value function is given by

$$
\begin{align}
V(t,x) &= \sup_{\pi} V^{\pi}(t,x). \notag 
\end{align}
$$



  ❓ **Question 3.1**:  Given a policy $\pi$, recall the Bellman relation for $V^{\pi}$ and the Bellman optimality principle for $V$.

  ❓ **Question 3.2**:  Apply Itô's formula to the process $V^{\pi}(s,X_s)$ between $t$ and $t+h$ for $h > 0$ and show that $V^{\pi}$ satisfies the   following linear PDE:

$$
\begin{align}
\begin{cases}
\frac{\partial V^{\pi}}{\partial t} (t,x) + \int_{A} \big[ H(x,a, \nabla_x V^{\pi}(t,x), D_x^2 V^{\pi}(t,x)) - \lambda \text{log}(\pi(t,x,a)) \big] \pi(t,x,a)\nu(da) &= 0,\notag \\
V^{\pi}(T,x) &= g(x),
\end{cases}
\end{align}
$$
where the map $H$ is defined as
$$
\begin{align}
H(x,a,p,M) = b(x,a) \cdot p + \frac{1}{2} \text{tr}(\sigma \sigma^{\top}(x,a) M) + f(x,a), \notag
\end{align}
$$
for $x \in \mathbb{R}^d$, $a \in A$, $p \in \mathbb{R}^d$, and $M \in \mathbb{R}^{d \times d}$.


❓ **Question 3.3**:  Deduce the Bellman equation satisfied by the optimal value function $V$:

$$
\begin{align}
\begin{cases}
\frac{\partial V}{\partial t} (t,x) + \underset{\pi \in \mathcal{P}(A)}{\text{ sup }} \int_{A} \big[ H(x,a, \nabla_x V^{\pi}(t,x), D_x^2 V^{\pi}(t,x)) - \lambda \text{log}(\pi(t,x,a)) \big] \pi(t,x,a)\nu(da) &= 0,\notag \\
V(T,x) &= g(x),
\end{cases}
\end{align}
$$

❓ **Question 3.4**:  Recall from the course the form of the optimal randomized policy $\pi^{\star}$ in terms of the value function $V$ and show that it leads to the following form of the Bellman equation for $V$:

$$
\begin{align}
\begin{cases}
\frac{\partial V}{\partial t} (t,x) + \lambda \text{ log } \bigg[ \int_{A} \text{ exp } (   \frac{1}{\lambda} H(x,a, D_x V(t,x)),D^2_x V(t,x)) \nu(d a ) \bigg] &= 0,\notag \\
V(T,x) &= g(x),
\end{cases}
\end{align}
$$



<a id=Mathematical-Questions-LQ-case></a>

<h2> Linear quadratic case in continuous time : </h2>



Suppose that the coefficients of the state dynamics and the reward functions are given by
$$\begin{align}
\begin{cases}
b(x,a) &= Bx + Ca, \notag \\
\sigma(x,a) &= Dx + Fa, \notag \\
f(x,a) &=  x^{\top} Q x + a^{\top} N a , \notag \\
g(x) &= x^{\top} P x, \notag
\end{cases}
\end{align}
$$
with $x \in \mathbb{R}^d$, $a \in \mathbb{R}^m$, and where $B$, $C$, $D$, $F$, $Q$, $N$, and $P$ are matrices of appropriate dimensions, with $Q$, $N$, and $P$ symmetric positive definite.


❓ **Question 3.5**:  Make the ansatz that the optimal value function is quadratic in the state variable, ie., of the form

$$
\begin{align}
V(t,x) = x^{\top} K(t) x + R(t), \notag 
\end{align}
$$
for some deterministic functions $K : [0,T] \rightarrow \mathbb{S}_{+}^d$ and $R : [0,T] \rightarrow \mathbb{R}$. 

- Show that the map $H$ is given by the form in the course (Slide 59 of Lecture 1).
- Show using Question 3.1.4 that the Bellman equation for $V$ is satisfies if $K$ and $R$ satisfy the system of ODEs given in the course (Slide 60 of Lecture 1).
- Show that the optimal feedback control policy $\pi^{\star}$ is given by a Gaussian distribution with mean and covariance matrix given in the course (Slide 60 of Lecture 1). Discuss the impact of $\lambda$ on the optimal policy and compare it to the case $\lambda = 0$ (without entropy regularization).



<a id=Mathematical-Questions></a>


<a id=references></a>

<h2> <center> 4. References   </center> </h2>





- R. Sutton and A. Barto: Introduction to reinforcement learning, second edition 2016, available [here](https://web.stanford.edu/class/psych209/Readings/SuttonBartoIPRLBook2ndEd.pdf).

- Y. Jia and X.Y. Zhou: Policy gradient and Actor-Critic learning in continuous time and space: theory and algorithms, 2022, Journal of Machine Learning and Research. available [here](https://arxiv.org/abs/2111.11232).

-  Y. Jia and X.Y. Zhou: q-Learning in continuous time, 2023, Journal of Machine Learning and Research. available [here](https://arxiv.org/abs/2207.00713).



